In [32]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import os
import sys
import re
from argparse import Namespace

sys.path.append('../src-py/')

In [34]:
%autoreload

from utils import *
from generate_conclusions import *

In [35]:
import torch
import json

import nltk
import numpy as np
import pandas as pd

from pathlib import Path
from datasets import load_dataset, load_metric, Dataset

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import BartTokenizer, BartForConditionalGeneration

In [36]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [37]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [7]:
ceph_dir = '/home/sile2804/data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation'
local_home_dir = '../data'

In [19]:
#Evalute ROUGE scores
gen_kwargs = {
    "do_sample": True, 
    "max_length":200,
    "top_p":0.95, 
    "top_k":50,
}

def generate_ds_attacks(ds, model, tokenizer, premises_clm, conclusion_clm, gen_kwargs, skip_special_tokens=True):
    ds = ds.map(lambda x :preprocess_function(x, tokenizer, premises_clm, 'counter', conclusion_clm=conclusion_clm), batched=True)
    ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    dataloader = torch.utils.data.DataLoader(ds, batch_size=8)
    attacks = generate_counters(model, tokenizer, dataloader, gen_kwargs, skip_special_tokens=skip_special_tokens)
    
    return attacks

#### Load models and testing data:

In [8]:
#masked_conclusion_tokenizer = BartTokenizer.from_pretrained(local_home_dir + '/output/extended_tokenizer')
#masked_conclusion_model = BartForConditionalGeneration.from_pretrained(local_home_dir + '/output/masked-conclusion-bart-model/').to(device)

known_conclusion_model = BartForConditionalGeneration.from_pretrained(local_home_dir  + '/output/known-conclusion-bart-model').to(device)
known_conclusion_tokenizer = BartTokenizer.from_pretrained(local_home_dir + '/output/extended_tokenizer')

pred_conclusion_model = BartForConditionalGeneration.from_pretrained(local_home_dir  + '/output/pred-conclusion-bart-model/').to(device)
pred_conclusion_tokenizer = BartTokenizer.from_pretrained(local_home_dir + '/output/pred-conclusion-bart-model/')

In [8]:
#KIALO models
masked_conclusion_kialo_model = BartForConditionalGeneration.from_pretrained(local_home_dir + '/output/masked-conclusion-kialo-model/').to(device)
masked_conclusion_kialo_tokenizer = BartTokenizer.from_pretrained(local_home_dir + '/output/masked-conclusion-kialo-model/')
known_conclusion_kialo_model = BartForConditionalGeneration.from_pretrained(local_home_dir + '/output/known-conclusion-kialo-model/').to(device)
known_conclusion_kialo_tokenizer = BartTokenizer.from_pretrained(local_home_dir + '/output/known-conclusion-kialo-model/')

#masked_conclusion_kialo_model = BartForConditionalGeneration.from_pretrained(local_home_dir + '/output/masked-conclusion-kialo-unique-model/').to(device)
#known_conclusion_kialo_model = BartForConditionalGeneration.from_pretrained(local_home_dir + '/output/known-conclusion-kialo-unique-model/').to(device)

##### Loading validation data from Reddit:

In [39]:
valid_ds = Dataset.from_pandas(pd.read_pickle(ceph_dir + '/reddit_data/conclusion_and_ca_generation/valid_conclusion_comp_remove_75sem_perc.pkl'))

tmp_ds = valid_ds.train_test_split(0.05)
sample_valid_ds = tmp_ds['test']

#saving the sample
reddit_sample_valid_ds = sample_valid_ds.flatten_indices()
reddit_sample_valid_ds.save_to_disk('../data/sample_valid_ds')

  0%|          | 0/2 [00:00<?, ?ba/s]

In [9]:
#loading the random sample of validation dataset with the auto-generated conclusion
reddit_sample_valid_ds = Dataset.load_from_disk('../data/sample_valid_ds/')

In [ ]:
#adding auto-conclusions from bart to the dataset
bart_conclusions = [generate_conclusion(p) for p in reddit_sample_valid_ds['masked_premises']]
reddit_sample_valid_ds = reddit_sample_valid_ds.add_column('bart_conclusion', bart_conclusions)

Your max_length is set to 62, but you input_length is only 52. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 62, but you input_length is only 28. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 62, but you input_length is only 54. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 62, but you input_length is only 57. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 62, but you input_length is only 41. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 62, but you input_length is only 48. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 62, but you input_length is only 57. You might consider decrea

##### Loading validation data from Kialo:

In [29]:
valid_ds = Dataset.from_pandas(pd.read_pickle(ceph_dir + '/kialo_data/kialo_valid_df.pkl'))

In [10]:
tmp_ds = valid_ds.train_test_split(0.5)
sample_valid_ds = tmp_ds['test']

#saving the sample
sample_valid_ds = sample_valid_ds.flatten_indices()
sample_valid_ds.save_to_disk('../data/kialo_sample_valid_ds')

  0%|          | 0/3 [00:00<?, ?ba/s]

In [9]:
sample_valid_ds = Dataset.load_from_disk('../data/kialo_sample_valid_ds')

### Evaluating on Kialo:

In [11]:
known_conc_attacks = generate_ds_attacks(sample_valid_ds, known_conclusion_kialo_model, known_conclusion_kialo_tokenizer, 'premises', 'conclusion_text', gen_kwargs)
masked_conc_attacks = generate_ds_attacks(sample_valid_ds, masked_conclusion_kialo_model, masked_conclusion_kialo_tokenizer, 'premises', None, gen_kwargs)
#auto_conc_attacks = generate_ds_attacks(sample_valid_ds, known_conclusion_model, tokenizer, 'masked_premises', 'auto_conclusion', gen_kwargs)

  0%|          | 0/3 [00:00<?, ?ba/s]

/usr/local/lib/python3.6/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


  0%|          | 0/3 [00:00<?, ?ba/s]

In [12]:
kialo_pred_df = pd.DataFrame(list(zip(sample_valid_ds['conclusion_text'], sample_valid_ds['premises'], 
                                   sample_valid_ds['counter'], known_conc_attacks, masked_conc_attacks)), 
                      columns=['conclusion', 'premises', 'gt', 'known_conc_attacks', 'masked_conc_attacks'])

kialo_pred_df['argument'] = kialo_pred_df.apply(lambda row: row['conclusion'] + ' : ' + ' '.join(row['premises']), axis=1)
kialo_pred_df['premises'] = kialo_pred_df['premises'].apply(lambda x: ' '.join(x))

In [13]:
kialo_pred_df[['argument', 'gt', 'known_conc_attacks', 'masked_conc_attacks']].sample(10).head(n=10)

,argument,gt,known_conc_attacks,masked_conc_attacks
404,"In some German states it is forbidden to host dance events and concerts on certain holidays. : This leads to a decrease in happiness and leisure activities for non-Christians. These bans have existed in Germany since at least the early 20th century Dance events and concerts contribute to the economy as food and beverages are generally consumed in large quantities, and tickets normally being required for attendance.","The German dance scene is similar to dance scenes in other countries where it is allowed to host dance events on Easter. Thus, no major economic effect is visible.","Some German states have banned cultural events and concerts on certain holidays, such as Christmas Eve and New Year's Eve.","Dance events and concerts contribute to the economy as food and beverages are generally consumed in large quantities, and tickets normally being required for attendance."
4,"When one person is having sex only to get paid, that sex is not fully consensual and so should not be allowed or encouraged. : The sex industry is often a means of providing for oneself or a family when all other options have been exhausted. Choice is always contingent on circumstances; the most free choices are made when all options are equally viable. For sex workers, choosing not to have sex is not a viable choice because it directly undermines their livelihood. When many sex workers have a history of abuse, you cannot assume that consent can be given. Monetary exchange is an inequitable power dynamic that undermines free choice. We should not permit sexual activity that does not proceed from personal relationships that exist on a level field.","Under this logic, there are a series of jobs that should simply be banished, like mining, for instance: it's something miners are doing only to get paid, so taking action on the job is not fully consensual, and would thus configure slavery, and should not be allowed or encouraged.\nThe same could be said of countless jobs.","The sex industry is often a means of providing for oneself or a family when all other options have been exhausted. Choice is always contingent on circumstances; the most free choices are made when all options are equally viable. For sex workers, choosing not to have sex is not a viable choice because it directly undermines their livelihood.","The sex industry is often a means of providing for oneself or a family when all other options have been exhausted. Choice is always contingent on circumstances; the most free choices are made when all options are equally viable. For sex workers, choosing not to have sex is not a viable choice because it directly undermines their livelihood. When many sex workers have a history of abuse, you cannot assume that consent can be given. Monetary exchange is an inequitable power dynamic that undermines free choice."
615,"Judge by Intent : Friends gained in by judging their intent will want to be better with you Intention allows for failure in action that may not always be possible; if a friend truly cares for you, their sentiment is more important than their mistakes. The action is observable, so the judging actually occurs when we examine intent. Intent is hard to establish, because we rely on asking the offender to explain why they did what they did. We use clues, such as intonation of voice, expression of face, choice that led to action, previous actions etc, to determine the motivation and to judge the character and intent. The action is observable and a fact of occurence, not needing judgement.",Friends gained by intent may fail at certain actions causing complications to both lives,"Intention is hard to establish, because we rely on asking the offender to explain why they did what they did. We use clues, such as intonation of voice, expression of face, choice that led to action, previous actions etc, to determine the motivation and to judge the character and intent. The action is observable and a fac

### Evaluating on Reddit:

In [13]:
# known_conc_attacks  = generate_ds_attacks(reddit_sample_valid_ds, known_conclusion_model, known_conclusion_tokenizer, 'masked_premises', 'title', gen_kwargs)
# masked_conc_attacks = generate_ds_attacks(reddit_sample_valid_ds, masked_conclusion_model, masked_conclusion_tokenizer, 'masked_premises', None, gen_kwargs)
# auto_conc_attacks = generate_ds_attacks(reddit_sample_valid_ds, known_conclusion_model, known_conclusion_tokenizer, 'masked_premises', 'auto_conclusion', gen_kwargs)
# pred_conc_attacks = generate_ds_attacks(reddit_sample_valid_ds, pred_conclusion_model, pred_conclusion_tokenizer, 'masked_premises', None, gen_kwargs, skip_special_tokens=False)

Loading cached processed dataset at ../data/sample_valid_ds/cache-b6d73bd865c592fa.arrow
/usr/local/lib/python3.6/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
Loading cached processed dataset at ../data/sample_valid_ds/cache-c848f93cbdfc7b58.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
known_conc_attacks  = generate_ds_attacks(reddit_sample_valid_ds, known_conclusion_model, known_conclusion_tokenizer, 'masked_premises', 'title', gen_kwargs)
bart_conc_attacks   = generate_ds_attacks(reddit_sample_valid_ds, known_conclusion_model, known_conclusion_tokenizer, 'masked_premises', 'bart_conclusion', gen_kwargs)
masked_conc_attacks = generate_ds_attacks(reddit_sample_valid_ds, known_conclusion_model, known_conclusion_tokenizer, 'masked_premises', None, gen_kwargs)
#This model would predict the conclusion and then the counter in one sequence.
joint_conc_attacks  = generate_ds_attacks(reddit_sample_valid_ds, pred_conclusion_model, pred_conclusion_tokenizer, 'masked_premises', None, gen_kwargs, skip_special_tokens=False)

In [ ]:
reddit_pred_df = pd.DataFrame(list(zip(reddit_sample_valid_ds['title'], 
                                       reddit_sample_valid_ds['bart_conclusion'], 
                                       reddit_sample_valid_ds['masked_premises'], 
                                       reddit_sample_valid_ds['counter'], 
                                       known_conc_attacks, masked_conc_attacks, 
                                       bart_conc_attacks, joint_conc_attacks)), 
                columns=['conclusion', 'bart_conclusion', 'premises', 'gt_attack', 'known_conc_attacks', 
                         'masked_conc_attacks', 'bart_conc_attacks', 'joint_conc_attacks'])

reddit_pred_df['argument'] = reddit_pred_df.apply(lambda row: row['conclusion'] + ' : ' + ' '.join(row['premises']), axis=1)
reddit_pred_df['premises'] = reddit_pred_df['premises'].apply(lambda x: ' '.join(x))

#process the jointly generated conclusion and counter
reddit_pred_df['joint_conc'] = reddit_pred_df['joint_conc_attacks'].apply (lambda x: x.split('<counter>')[0])
reddit_pred_df['joint_conc_attacks'] = reddit_pred_df['joint_conc_attacks'].apply (lambda x: x.split('<counter>')[1] if '<counter>' in x else x)
reddit_pred_df['joint_conc'] = reddit_pred_df['joint_conc'].apply (lambda x: re.sub('<s>|</s>|<conclusion>|<counter>|<pad>', '', x).strip())
reddit_pred_df['joint_conc_attacks'] = reddit_pred_df['joint_conc_attacks'].apply (lambda x: re.sub('<s>|</s>|<conclusion>|<counter>|<pad>', '', x).strip())

In [48]:
reddit_pred_df[['argument', 'conclusion' , 'joint_conc', 'masked_conc_attacks', 'known_conc_attacks', 'joint_conc_attacks']].sample(10).head(n=10)

argument  \
269                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [49]:
reddit_pred_df.to_pickle('../data/output/reddit_pred_df.pkl')

In [8]:
reddit_pred_df = pd.read_pickle('../data/output/reddit_pred_df.pkl')

In [51]:
#collect references
valid_df = pd.read_pickle(ceph_dir + '/reddit_data/conclusion_and_ca_generation/valid_conclusion_comp_remove_75sem_perc.pkl')
valid_df['argument'] = valid_df.apply(lambda row: row['title'] + ' : ' + ' '.join(row['masked_premises']), axis=1)
arg_counters = valid_df.groupby('argument').agg({
    'counter': lambda x: [' '.join(c) for c in x]
}).reset_index()
arg_counters = pd.Series(arg_counters.counter.values, index=arg_counters.argument).to_dict()

In [52]:
reddit_pred_df['gt_attack'] = reddit_pred_df['gt_attack'].apply(lambda x: str(x))
reddit_pred_df['all_counters'] = reddit_pred_df['argument'].apply(lambda x: arg_counters[x])

In [53]:
masked_conc_eval = evaluate_gen_attacks(reddit_pred_df['masked_conc_attacks'].tolist(), reddit_pred_df['all_counters'].tolist())
known_conc_eval = evaluate_gen_attacks(reddit_pred_df['known_conc_attacks'].tolist(), reddit_pred_df['all_counters'].tolist())
bart_conc_eval = evaluate_gen_attacks(reddit_pred_df['bart_conc_attacks'].tolist(), reddit_pred_df['all_counters'].tolist())
pred_conc_eval = evaluate_gen_attacks(reddit_pred_df['joint_conc_attacks'].tolist(), reddit_pred_df['all_counters'].tolist())

In [55]:
from tabulate import tabulate

print(tabulate([['Masked Conclusion'] + [round(masked_conc_eval['bleu'], 2), masked_conc_eval['bert-fscore']],
                ['BART Conclusion'] + [round(bart_conc_eval['bleu'], 2), bart_conc_eval['bert-fscore']],
                ['Joint Prediction'] + [round(pred_conc_eval['bleu'], 2), pred_conc_eval['bert-fscore']],
                ['Known Conclusion'] + [round(known_conc_eval['bleu'], 2), known_conc_eval['bert-fscore']],
    ], headers=['bleu', 'bert-f1score']))

                     bleu    bert-f1score
-----------------  ------  --------------
Masked Conclusion    0.24            0.16
BART Conclusion      0.22            0.16
Joint Prediction     0.24            0.18
Known Conclusion     0.27            0.2


##### Similarity to conclusion:

In [15]:
masked_conc_eval = evaluate_gen_attacks(reddit_pred_df['conclusion'], reddit_pred_df['masked_conc_attacks'].tolist())
known_conc_eval  = evaluate_gen_attacks(reddit_pred_df['conclusion'], reddit_pred_df['known_conc_attacks'].tolist())
#auto_conc_eval   = evaluate_gen_attacks(reddit_pred_df['conclusion'], reddit_pred_df['auto_conc_attacks'].tolist())
pred_conc_eval   = evaluate_gen_attacks(reddit_pred_df['conclusion'], reddit_pred_df['pred_conc_attacks'].tolist())
gt_attack_conc_eval = evaluate_gen_attacks(reddit_pred_df['conclusion'], reddit_pred_df['gt'].tolist())

In [17]:
from tabulate import tabulate

print(tabulate([['masked'] + [round(masked_conc_eval['bleu'], 2), masked_conc_eval['bert-fscore']],
                #['auto'] + list(auto_conc_eval.values()),
                ['pred'] + [round(pred_conc_eval['bleu'], 2), pred_conc_eval['bert-fscore']],
                ['known'] + [round(known_conc_eval['bleu'], 2), known_conc_eval['bert-fscore']],
                ['gt_attack'] + [round(gt_attack_conc_eval['bleu'], 2), gt_attack_conc_eval['bert-fscore']]
    ], headers=['bleu', 'bert-f1score']))

             bleu    bert-f1score
---------  ------  --------------
masked       0               0.16
pred         0.01            0.18
known        0.03            0.33
gt_attack    0              -0.03
